Получите данные по безработице в Москве:
https://video.ittensive.com/python-advanced/data-9753-2019-07-25.utf.csv

Объедините эти данные индексами (Месяц/Год) с данными из предыдущего задания (вызовы пожарных) для Центральный административный округ:
https://video.ittensive.com/python-advanced/data-5283-2019-10-04.utf.csv

Найдите значение поля UnemployedMen в том месяце, когда было меньше всего вызовов в Центральном административном округе.

In [3]:
import pandas as pd
data_unemployment = pd.read_csv("https://video.ittensive.com/python-advanced/data-9753-2019-07-25.utf.csv", delimiter=";")
data_firefighters = pd.read_csv("https://video.ittensive.com/python-advanced/data-5283-2019-10-04.utf.csv", delimiter=";")
# из первого и второго набора данных удалим не нужные столбцы
data_unemployment = data_unemployment.drop(labels=["global_id", "ID", "Unnamed: 9", "UnemployedWomen", "UnemployedYoung", "UnemployedDisabled", "UnemployedTotal" ], axis=1)
data_firefighters = data_firefighters.drop(labels=["ID", "global_id", "Unnamed: 6"], axis=1)
# переименуем колонки в наборах данных
data_unemployment.columns = ["Год","UnemployedMen", "Месяц" ]
data_firefighters.columns = ["Округ","Год", "Месяц", "Звонки" ]

area_indexes = data_firefighters[data_firefighters["Округ"].str.contains("Центральный")].index  # создадим список номеров индекса центрального округа
#print(area_indexes)

# создадим списк занчений округа (или центарльный или другой) по ранее созданному списку индексов центрального округа
data_areas_index = [(lambda n:"Центральный" if n in area_indexes else "Другой")(i) for i in range(0, len(data_firefighters.index))]
#print(data_areas_index)

data_firefighters["Тип"] = data_areas_index  # добавялем в наш фреймданных еще одну серию и скажем, что это индекс наших зон

data_firefighters = data_firefighters.set_index(["Тип", "Округ"])  # после этого назначем уже мульти индекс
#data_firefighters = data_firefighters.sort_index() # отсортировали данные по индексам
areas = data_firefighters.loc["Центральный"]  # срез данных по значению мульти индекса
data = pd.merge(data_unemployment, areas)  # объединим данные по индексам

min_Calls = data[data['Звонки']==data['Звонки'].min()]  # найдем минимальное количество вызовов

print(min_Calls)
# print(data)

    Год  UnemployedMen    Месяц  Звонки
1  2015          13465  Февраль     220


Более простое решение от ittensive

In [20]:
import pandas as pd
data_1 = pd.read_csv("https://video.ittensive.com/python-advanced/data-9753-2019-07-25.utf.csv", delimiter=";")
data_1 = data_1.set_index(["Year", "Period"]) #Создаем множественные индексы
#print(data_1.head)
data_2 = pd.read_csv("https://video.ittensive.com/python-advanced/data-5283-2019-10-04.utf.csv", delimiter=";")
data_2 = data_2.set_index(["AdmArea", "Year", "Month"]) #Создаем множественные индексы
# Индекс округ создали для того, чтобы выбрать множественные занчения 
data_2 = data_2.loc["Центральный административный округ"]
#print(data_2.head)
#Для того, чтобы обьеденить колонки по индексам, нужно чтобы у них было одинаковое название.
# скажем, что теперь индексы у data_2 будут "Year", "Period" через свойства index.name
data_2.index.names = ["Year", "Period"]
#После этого можно будет спокойно произвести объединение по индексам
data = pd.merge(data_1, data_2, left_index=True, right_index=True)
# для того чтобы найти значение поля UnemployedMen в том месяце, когда было меньше всего вызовов
# нужно отбросить тек. индекс, он свою задачу решил, по нему объеденили данные, и назначить новый индекс по кол-ву вызовов
data = data.reset_index() # Сбросили индекс
data = data.set_index("Calls") # назначили новый
data = data.sort_index() # отсортировали по индексу (по возраст.)
#print(data.head) # выведя шапку уже получим результат.
#print(data[0:1]) # выведем только 1ю строку (так как осорт по возраст. выз.) и получим нужный результат.
print(data["UnemployedMen"].iloc[0:1]) # значение только UnemployedMen, при минимальном кол-ве вызовов

Calls
220    13465
Name: UnemployedMen, dtype: int64
